# A LangChain 1.0 RAG Agent that can talk with a PDF Document... properly

## What is RAG (Retrieval-Augmented Generation)?

**RAG** combines two powerful techniques:
1. **Retrieval**: Finding relevant information using semantic search (like we did in `020-pdf-agent.ipynb`)
2. **Generation**: Using an LLM to read that information and generate a coherent, natural language answer

**Why is this code an example of a RAG app?**
- It **retrieves** relevant document chunks using semantic search (via the tool)
- It **augments** the agent's knowledge with that retrieved context
- It **generates** a natural language answer by reading and understanding the retrieved text

This is different from just semantic search, which only gives you raw chunks without interpretation.

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("gen-ai-in-2026.pdf")

data = loader.load()

from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)

all_splits = text_splitter.split_documents(data)

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

ids = vector_store.add_documents(documents=all_splits)

In [3]:
from langchain.tools import tool

@tool
def search_handbook(query: str) -> str:
    """Search the handbook for information"""
    results = vector_store.similarity_search(query)
    return results[0].page_content

from langchain.agents import create_agent

agent = create_agent(
    model="gpt-4o-mini",
    tools=[search_handbook],
    system_prompt="You are a helpful agent that can search the pdf file for information."
    )

from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="According to Gartner, what percentage of enterprises will use Generative AI APis or deploy generative AI-enabled applications in production environments in 2026?")]}
)

print(response["messages"][-1].content)

According to Gartner, by 2026, more than 80% of enterprises will use generative AI APIs or deploy generative AI-enabled applications in production environments.


## Let's explain the previous code in simple terms

#### Step 1: Preparing the Data (Same as Before)

This part is identical to `020-pdf-agent.ipynb` - we're preparing our knowledge base:

```python
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("gen-ai-in-2026.pdf")
```
**Load the PDF file** - Points to the document we want to query.

```python
data = loader.load()
```
**Extract all pages** - Converts the PDF into Document objects.

```python
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
```
**Configure the text splitter** - Will break the document into 1000-character chunks with 200-character overlap.

```python
all_splits = text_splitter.split_documents(data)
```
**Split the documents** - Creates smaller, searchable chunks.

```python
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
```
**Create embeddings model** - This will convert text to numerical vectors that capture semantic meaning.

```python
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)
```
**Create vector database** - A specialized database for storing and searching vector embeddings.

```python
ids = vector_store.add_documents(documents=all_splits)
```
**Store all chunks** - Each chunk is converted to a vector and stored for later retrieval.

### Step 2: Creating the RAG Agent (The Magic Part!)

This is where RAG happens - we give an AI agent the ability to search the document and generate intelligent answers.

---

#### Line-by-Line Explanation

```python
from langchain.tools import tool
```
**Import the tool decorator** - This allows us to create "tools" that the agent can use. Tools are functions the agent can call when needed.

```python
@tool
def search_handbook(query: str) -> str:
    """Search the handbook for information"""
    results = vector_store.similarity_search(query)
    return results[0].page_content
```
**Create a search tool**:
- `@tool` decorator: Converts this function into a tool the agent can use
- The **docstring** (`"""Search the handbook for information"""`) is CRITICAL - the agent reads this to understand what the tool does
- `query: str`: The search query (the agent will generate this automatically based on the user's question)
- `vector_store.similarity_search(query)`: Performs semantic search to find relevant chunks
- `results[0].page_content`: Returns just the text content of the most relevant chunk
- **Why this works**: The agent can call this function whenever it needs information from the PDF

```python
from langchain.agents import create_agent
```
**Import the agent creator** - This is LangChain 1.0's primary way to build agents.

```python
agent = create_agent(
    model="gpt-4o-mini",
    tools=[search_handbook],
    system_prompt="You are a helpful agent that can search the pdf file for information."
)
```
**Create the RAG agent**:
- `model="gpt-4o-mini"`: The LLM that will power the agent's reasoning and text generation
- `tools=[search_handbook]`: Gives the agent access to our search tool
- `system_prompt`: Instructions that tell the agent its purpose and capabilities
- **What the agent can do**: 
  - Decide when to search the PDF (it won't search for "Hello" or simple questions)
  - Generate appropriate search queries
  - Read the retrieved content
  - Formulate natural language answers
  - Handle follow-up questions

```python
from langchain.messages import HumanMessage
```
**Import message types** - Agents work with structured messages.

```python
response = agent.invoke(
    {"messages": [HumanMessage(content="According to Gartner, what percentage of enterprises will use Generative AI APis or deploy generative AI-enabled applications in production environments in 2026?")]}
)
```
**Send a question to the agent**:
- `HumanMessage`: Represents a message from the user
- `agent.invoke()`: Runs the agent with the conversation
- **What happens internally**:
  1. Agent reads the question
  2. Agent decides it needs to search the PDF
  3. Agent generates a good search query: `"Gartner percentage enterprises Generative AI APIs 2026"`
  4. Agent calls the `search_handbook` tool
  5. Agent receives the retrieved chunk
  6. Agent reads and understands the chunk
  7. Agent extracts the answer and formulates a response

```python
print(response["messages"][-1].content)
```
**Print the final answer**:
- `response["messages"]`: Contains the full conversation (user question, tool calls, tool results, agent's answer)
- `[-1]`: Gets the last message (the agent's final answer)
- `.content`: Gets just the text of that message

**Output**: `"According to Gartner, by 2026, more than 80% of enterprises will use generative AI APIs or deploy generative AI-enabled applications in production environments."`

---

#### How the Agent Makes Decisions

The agent uses a **ReAct pattern** (Reasoning + Acting):

1. **Reasoning**: "The user is asking about a Gartner statistic. I need to search the document."
2. **Acting**: Calls `search_handbook("Gartner percentage enterprises Generative AI APIs 2026")`
3. **Observing**: Reads the retrieved chunk
4. **Reasoning**: "I found the answer in the text: 'more than 80% of enterprises will use generative AI APIs'"
5. **Acting**: Generates the final answer in natural language

The agent can:
- Skip searches for simple questions it can answer directly
- Generate multiple search queries if needed
- Refine searches if the first result isn't good enough
- Combine information from multiple chunks (if configured to do so)

In [4]:
from pprint import pprint

pprint(response['messages'])

[HumanMessage(content='According to Gartner, what percentage of enterprises will use Generative AI APis or deploy generative AI-enabled applications in production environments in 2026?', additional_kwargs={}, response_metadata={}, id='d9c5196d-baa1-46ab-917b-68abb3ad2f0c'),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 88, 'total_tokens': 116, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'id': 'chatcmpl-CxtLaahfUsKCIW8U4xrbZveUVrGe8', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bbc44-27b6-7e13-a73c-8693a26a390a-0', tool_calls=[{'name': 'search_handbook', 'args': {'query': 'Gartner perce

### Understanding the Message Flow

The `pprint(response['messages'])` output shows the complete conversation between components:

1. **HumanMessage**: Your original question
2. **AIMessage #1**: The agent deciding to use the tool (contains `tool_calls`)
3. **ToolMessage**: The result from `search_handbook` (the retrieved chunk)
4. **AIMessage #2**: The agent's final answer after reading the tool result

This transparency is valuable for debugging and understanding how the agent works.

## RAG vs Semantic Search: Why RAG Gives Better Responses

### Comparing the Two Approaches

| Aspect | **Semantic Search** (020-pdf-agent.ipynb) | **RAG Agent** (021-rag-agent.ipynb) |
|--------|------------------------------------------|-------------------------------------|
| **What you get** | Raw document chunks with metadata | Natural language answers |
| **LLM involved?** | No - just vector similarity | Yes - agent reads and understands |
| **Output format** | Technical Document object | Human-friendly text |
| **Answer extraction** | Manual - you read the chunk | Automatic - agent extracts the answer |
| **Multiple chunks** | Only shows one (unless you loop) | Agent can synthesize information from context |
| **Follow-up questions** | No memory or context | Agent can handle conversational flow |
| **Decision making** | You decide when to search | Agent decides when retrieval is needed |
| **User experience** | Poor - requires technical knowledge | Excellent - like talking to an expert |

---

### Example Comparison

#### Semantic Search Output (020-pdf-agent.ipynb):
```
page_content='Generative AI in 2026:\nTransforming Business and\n Professional Value\nAs we progress through 2026, Generative AI has reached a critical inflection point...'
metadata={'producer': 'ReportLab PDF Library', 'page': 0, 'page_label': '1', 'start_index': 0}
```
**Problems**:
- You have to manually read the chunk to find "80%"
- Includes irrelevant metadata
- Shows more text than needed
- Not conversational or user-friendly

#### RAG Agent Output (021-rag-agent.ipynb):
```
According to Gartner, by 2026, more than 80% of enterprises will use generative AI APIs 
or deploy generative AI-enabled applications in production environments.
```
**Benefits**:
- Direct, clear answer
- Extracted exactly what was asked
- Professional, natural language
- Ready to present to users

---

### Why RAG Gives Better Responses: The Key Differences

#### 1. **Intelligence Layer**
- **Semantic Search**: Dumb retrieval - finds similar text but doesn't understand what you need
- **RAG**: Smart retrieval + comprehension - understands your question, finds relevant info, AND interprets it

#### 2. **Answer vs Data**
- **Semantic Search**: Returns raw data chunks (like giving someone a whole page from an encyclopedia)
- **RAG**: Returns precise answers (like having an expert read the page and tell you exactly what you need)

#### 3. **Contextual Understanding**
- **Semantic Search**: No understanding of what information is relevant in the chunk
- **RAG**: LLM reads the chunk, understands it, and extracts only the relevant information

#### 4. **Multi-step Reasoning**
- **Semantic Search**: One search, one result, done
- **RAG Agent**: Can chain multiple searches, reason about results, and synthesize information

#### 5. **Handling Complexity**
Example question: *"Compare what Gartner and MIT said about AI adoption"*
- **Semantic Search**: Would return one chunk (either Gartner OR MIT stats)
- **RAG Agent**: Could search for Gartner stats, then search for MIT stats, then compare them

#### 6. **User Experience**
- **Semantic Search**: Requires user to be technical, read through chunks, find answers
- **RAG Agent**: Works like ChatGPT - natural conversation, clean answers

---

### The Power of Agentic RAG

This implementation uses **Agentic RAG**, which means:

1. **Autonomous Decision-Making**: The agent decides WHEN to search (not every query needs retrieval)
   - "Hello" → No search needed
   - "What's 2+2?" → No search needed  
   - "What did Gartner say?" → Search needed

2. **Dynamic Query Generation**: The agent creates better search queries than you might
   - Your question: "According to Gartner, what percentage..."
   - Agent's search: "Gartner percentage enterprises Generative AI APIs 2026"
   - Result: More focused retrieval

3. **Multi-turn Interactions**: The agent can have conversations
   - User: "What did Gartner say about AI in 2026?"
   - Agent: *searches and answers*
   - User: "What about MIT?"
   - Agent: *searches again with new context*

4. **Tool Integration**: Agents can use multiple tools
   - Search tool (what we have)
   - Could add: web search, calculator, database queries, etc.

---

### Real-World Impact

**Why RAG matters in production**:
- Research shows RAG can improve answer accuracy by up to **70%**
- Users get ChatGPT-quality responses from your private documents
- No need to fine-tune expensive models on your data
- Documents can be updated without retraining
- Scales to millions of documents efficiently

**Use cases**:
- Customer support bots (search company knowledge base)
- Legal document analysis (find relevant case law)
- Medical literature review (find research papers)
- Enterprise documentation (internal wikis, handbooks)
- Educational tutors (textbook Q&A)

---

### Key Takeaway

**Semantic Search** is like having a library card catalog - it tells you which books might have your answer, but you still have to read them.

**RAG** is like having a research librarian - they find the relevant books, read them for you, and give you a clear, direct answer to your question.

**Agentic RAG** is like having a smart assistant - they decide when to use the library, what to search for, and can even combine information from multiple sources autonomously.

---

### Evolution Path

```
Simple Keyword Search → Semantic Search → RAG → Agentic RAG → Multi-Agent RAG
     (1990s)              (2020s)        (2023)    (2024-2026)      (Future)
```

We're currently in the **Agentic RAG era**, where agents make intelligent decisions about retrieval and reasoning.

## How to run this code from Visual Studio Code
* Open Terminal.
* Make sure you are in the project folder.
* Make sure you have the poetry env activated.
* Enter and run the following command:
    * `python 021-rag-agent.py`